# API 4-4: REST APIs 

Hosting your own APIs with FastAPI



## API vs Module 


You might be asking why host an API versus just writing a module? After all they have similar functionality of sharing and interacting with code and data.


        def do_something_sensitive():
        # do something sensitive
        # if I import this, I can see this code
        return "sensitive data"

        VS

        /api/do_something_sensitive


There are some key differences between an API and a module:

1. You can hide the implementation details of your code with an API. Unlike a module, people only see the interface, not way to code was implemented.
2. An API can be accessed over the internet, while a module is typically used locally.
3. API's work across different programming languages, while modules are restricted to the language they were written in (Python).
4. Cede is hosted as a web service, meaning it can be accessed from anywhere, anytime with an internet connection.


### Why API?

- Break up complex applications into smaller, more manageable pieces
- Allow different parts of an application to be developed independently
- Allow different parts of an application to be developed in different languages


## Fast API

[https://fastapi.tiangolo.com/](https://fastapi.tiangolo.com/)

- Fast API is a easy to learn Python framework for building API's. 
- It has a lot of features that make it a great choice for building API's, but 
- we will focus keeping it simple so that you can understand the benefits of building an API.


# Fast API Features

- Live edit similar to Streamlit. You can edit your code and see the changes in real time.
- Highly opinionated. It has a lot of features that are built in but you must do it the "FastAPI way."
- Easy to understand the code. 
- Auto-generates API documentation Swagger UI.

In [2]:
# fasthello.py

from fastapi import FastAPI

app = FastAPI()  # Create a FastAPI instance

@app.get("/")    # Define a route
def root():      # Define a function that will be called when the route is requested
    return {"message": "Hello World"} # Serializes to JSON automatically


## http://localhost:8000/docs

## Query String Path, and Header Parameters

Fast API makes it easy to add parameters to your API. 

- Python type hints are used to define the type of the parameter (int, str, etc.)
- The `Query()` function is used to define a query parameter
- The `Header()` function is used to define a header parameter
- Path parameters are defined by including the parameter in the URL path


In [ ]:
# fastparams.py

from fastapi import FastAPI, Header, Query

app = FastAPI()

@app.get("/calculator/{operator}")  # <== path parameter
def read_item(operator: str, 
              a: str = Query(),     # <== query parameter
              b: int = Query(),     # <== query parameter
              h: str = Header()):   # <== header parameter
    if operator == "add":
        result = a + b
    elif operator == "sub":
        result = a - b
    elif operator == "mul":
        result = a * b
    elif operator == "div":
        result = a / b
    return {
        "operator": operator,
        "a": a,
        "b": b,
        "result": result,
        "h": h
    }

## Challenge 4-4-1

Design and build an API to search for  flights (depart/arrive) by Airport Code. 

Use this dataset to get the source of your flights:

https://raw.githubusercontent.com/mafudge/datasets/refs/heads/master/flights/sample-flights.csv

Here's some examples of the the API endpoint you need to build:

```
/api/flights/search?type=dep&code=OKA
/api/flights/search?type=arr&code=KEY
```

Test your API using the Swagger UI.

## Handling Errors

FastAPI makes it easy to handle errors. Use the `HTTPException` class to raise an error with a specific status code and message.

Remember status codes are a way to communicate the status of a request to the client. They are a design contract which means you SHOULD use them as they are EXPECTED, but they are not REQURED.

[https://developer.mozilla.org/en-US/docs/Web/HTTP/Status](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status)


In [ ]:
# fastparams2.py

from fastapi import FastAPI, Header, Query, HTTPException

app = FastAPI()

@app.get("/calculator/{operator}")  # <== path parameter
def read_item(operator: str, 
              a: str = Query(),     # <== query parameter
              b: int = Query(),     # <== query parameter
              h: str = Header()):   # <== header parameter
    if operator == "add":
        result = a + b
    elif operator == "sub":
        result = a - b
    elif operator == "mul":
        result = a * b
    elif operator == "div":
        result = a / b
    else:
        raise HTTPException(status_code=404, detail="Operator not found. should be: add, sub, mul, div")
    
    return {
        "operator": operator,
        "a": a,
        "b": b,
        "result": result,
        "h": h
    }

## Sending data in the request body

- Use the `Body()` function to define a request body parameter
- These are added to POST/PUT requests in JSON format.

In [ ]:
# fastbody.py

from fastapi import FastAPI, Body()

app = FastAPI()

friends = []

@app.post("/friends")
def create_friend(name: str = Body(), age: int = Body()):
    '''
    Add a new friend to the list of friends and return the list each time a new friend is added
    ''' 
    friend = {"name": name, "age": age}
    friends.append(friend)
    return friends

## API Wrappers

A common use case for API's is to wrap an existing API to make it easier to use. The IoT Portal is a good example of this.

API Wrappers are common with AI/ML models. We want to control access and restrict behavior of the AI and the best way to do that is wrap a public AI API in your own API. 

## Challenge 4-4-2

TLDR (Too Long Didn't Read) bot. 

Let's design and build an API to wrap the OpenAI API in the IoT portal. We will build a tldr bot that summarizes a body of text in a short / simple way. 

Things to figure out:

- What should the endpoint name be?
- Method (GET, POST, PUT, DELETE)?
- What parameters should be passed?
- What should the response look like?
- What does the OpenAI API prompt look like? 
- What is the system prompt?
- Few shots?

NOTE: 


## Building a CRUD API

- CRUD stands for Create, Read, Update, Delete
- Its a common pattern for building non-public data APIs
- We use the HTTP methods POST (Create), GET (Read), PUT (Update), DELETE to perform these operations

**NOTE:** to perform CRUD, we need a business key or natural key to identify the item. 

In the example below, we use the `name`  of our friend. Not the best, but workable for our example. 

The best practice for API design is to return the object that was created, updated, or deleted. This allows the client to know the state of the object after the operation.

See `fastcrud.py` for an example of a CRUD API.

## TinyDb - Persistent CRUD Storage

TinyDB is a lightweight document oriented database optimized for your happiness. It's written in pure Python and has no external dependencies. 

It gives you CRUD over a JSON file.

[https://tinydb.readthedocs.io/en/latest/](https://tinydb.readthedocs.io/en/latest/)


In [9]:
from tinydb import TinyDB, Query

# Initialize TinyDB
db = TinyDB('db.json')
db.truncate()  #start over


# Create documents
print("CREATE Alice, Bob, Charlie")
db.insert({'name': 'Alice', 'age': 25, 'hometown': 'Seattle'})
db.insert({'name': 'Bob', 'age': 22, 'hometown': 'Portland'})
db.insert({'name': 'Charlie', 'age': 30, 'hometown': 'San Francisco'})

# get all people
print("PEOPLE", db.all())

# update bob
Person = Query()
print("UPDATE Bob")
db.update({'age': 23, 'hometown' : 'New York'}, Person.name == 'Bob')
print("PEOPLE", db.all())

# delete alice
print("DELETE Alice")
db.remove(Person.name == 'Alice')
print("PEOPLE", db.all())

# loop
print("PEOPLE")
for person in db:
    print(person['name'], person['age'], person['hometown'])



CREATE Alice, Bob, Charlie
PEOPLE [{'name': 'Alice', 'age': 25, 'hometown': 'Seattle'}, {'name': 'Bob', 'age': 22, 'hometown': 'Portland'}, {'name': 'Charlie', 'age': 30, 'hometown': 'San Francisco'}]
UPDATE Bob
PEOPLE [{'name': 'Alice', 'age': 25, 'hometown': 'Seattle'}, {'name': 'Bob', 'age': 23, 'hometown': 'New York'}, {'name': 'Charlie', 'age': 30, 'hometown': 'San Francisco'}]
DELETE Alice
PEOPLE [{'name': 'Bob', 'age': 23, 'hometown': 'New York'}, {'name': 'Charlie', 'age': 30, 'hometown': 'San Francisco'}]
PEOPLE
Bob 23 New York
Charlie 30 San Francisco


## Challenge 4-4-3

Putting it all together so far.

Take your TLDR bot from 4-4-2 and build a UI for it in streamlit. This can be a simple question / answer user interface. You do not need to make it conversational.